In [1]:
pip install evalml

  Using cached pandas-1.1.5-cp37-cp37m-win_amd64.whl (8.7 MB)

  Using cached lightgbm-3.0.0-py2.py3-none-win_amd64.whl (737 kB)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
automl 2.9.9 requires dill<0.3,>=0.2.5, but you have dill 0.3.3 which is incompatible.
automl 2.9.9 requires lightgbm<2.1,>=2.0.11, but you have lightgbm 3.0.0 which is incompatible.
automl 2.9.9 requires pandas<1.0,>=0.18.0, but you have pandas 1.1.5 which is incompatible.



  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.0.12
    Uninstalling lightgbm-2.0.12:
      Successfully uninstalled lightgbm-2.0.12


In [6]:
from urllib.request import urlopen
import pandas as pd

input_data = urlopen('https://featurelabs-static.s3.amazonaws.com/spam_text_messages_modified.csv')
data = pd.read_csv(input_data)

In [8]:
data.head()

,Category,Message
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,spam,FreeMsg Hey there darling it's been 3 week's n...
2,spam,WINNER!! As a valued network customer you have...
3,spam,Had your mobile 11 months or more? U R entitle...
4,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [9]:
x = data.drop(['Category'],axis=1)
y = data['Category']

In [11]:
y.value_counts(normalize=True)

ham     0.750084
spam    0.249916
Name: Category, dtype: float64

In [12]:
import evalml

In [13]:
features_train,features_test,target_train,target_test = evalml.preprocessing.split_data(x,y,problem_type='binary')

In [14]:
from evalml import  AutoMLSearch

In [16]:
automl = AutoMLSearch(X_train=features_train,y_train=target_train,problem_type='binary',max_batches=2,optimize_thresholds=True)

In [17]:
automl.search()

Numerical binary classification target classes must be [0, 1], got [ham, spam] instead
Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Searching up to 2 batches for a total of 14 pipelines. 
Allowed model families: catboost, xgboost, linear_model, lightgbm, extra_trees, decision_tree, random_forest



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/14) Mode Baseline Binary Classification P... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 8.638
Batch 1: (2/14) Logistic Regression Classifier w/ Imp... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.214
High coefficient of variation (cv >= 0.2) within cross validation scores. Logistic Regression Classifier w/ Imputer + Text Featurization Component + Standard Scaler may not perform as estimated on unseen data.
Batch 1: (3/14) Random Forest Classifier w/ Imputer +... Elapsed:00:23
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.155
High coefficient of variation (cv >= 0.2) within cross validation scores. Random Forest Classifier w/ Imputer + Text Featurization Component may not perform as estimated on unseen data.
Batch 1: (4/14) XGBoost Classifier w/ Imputer + Text ... Elapsed:00:41
	Starting cross validation
	Finished cross validation - mean Log Los

In [18]:
automl.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,2,Random Forest Classifier w/ Imputer + Text Fea...,0.154849,0.110302,98.207418,True,{'Imputer': {'categorical_impute_strategy': 'm...
4,3,XGBoost Classifier w/ Imputer + Text Featuriza...,0.178639,0.113254,97.932010,True,{'Imputer': {'categorical_impute_strategy': 'm...
5,1,Logistic Regression Classifier w/ Imputer + Te...,0.214011,0.165624,97.522538,True,{'Imputer': {'categorical_impute_strategy': 'm...
6,7,LightGBM Classifier w/ Imputer + Text Featuriz...,0.214580,0.136260,97.515944,True,{'Imputer': {'categorical_impute_strategy': 'm...
9,6,Extra Trees Classifier w/ Imputer + Text Featu...,0.252206,0.216198,97.080377,True,{'Imputer': {'categorical_impute_strategy': 'm...
10,4,CatBoost Classifier w/ Imputer + Text Featuriz...,0.526403,0.512717,93.906174,False,{'Imputer': {'categorical_impute_strategy': 'm...
11,5,Elastic Net Classifier w/ Imputer + Text Featu...,0.542803,0.529152,93.716325,False,{'Imputer': {'categorical_impute_strategy': 'm...
12,8,Decision Tree Classifier w/ Imputer + Text Fea...,0.801766,0.555179,90.718481,True,{'Imputer': {'categorical_impute_strategy': 'm...
13,0,Mode Baseline Binary Classification Pipeline,8.638305,8.623860,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}


In [20]:
best_pipeline  = automl.best_pipeline

In [22]:
automl.best_pipeline

GeneratedPipelineBinary(parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Text Featurization Component':{'text_columns': ['Message']}, 'Random Forest Classifier':{'n_estimators': 100, 'max_depth': 6, 'n_jobs': -1},})

In [21]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

**********************************************************************
* Random Forest Classifier w/ Imputer + Text Featurization Component *
**********************************************************************

Problem Type: binary
Model Family: Random Forest

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. Text Featurization Component
	 * text_columns : ['Message']
3. Random Forest Classifier
	 * n_estimators : 100
	 * max_depth : 6
	 * n_jobs : -1

Training
Training for binary problems.
Total training time (including CV): 17.6 seconds

Cross Validation
----------------
             Log Loss Binary  MCC Binary   AUC  Precision    F1  Balanced Accuracy Binary  Accuracy Binary # Training # Validation
0                      0.110       0.895 0.987      0.938 0.921                     0.942            0.961   1594.000      797.000
1                      0.144 

In [23]:
scores = best_pipeline.score(features_test,target_test,objectives = evalml.objectives.get_core_objectives('binary'))

In [25]:
print(f'Accuracy Binary: {scores["Accuracy Binary"]}')

Accuracy Binary: 0.9732441471571907


In [28]:
scores

OrderedDict([('MCC Binary', 0.9278003804626707),
             ('Log Loss Binary', 0.1137465525638786),
             ('AUC', 0.9823022077397945),
             ('Precision', 0.9716312056737588),
             ('F1', 0.9448275862068964),
             ('Balanced Accuracy Binary', 0.9552772006397513),
             ('Accuracy Binary', 0.9732441471571907)])

In [29]:
from evalml.utils import infer_feature_types

In [32]:
X = infer_feature_types(x, {'Message': 'Categorical'})

In [36]:
features_train, features_test, target_train, target_test = evalml.preprocessing.split_data(X, y, problem_type='binary', test_size=0.2)

In [37]:
automl_no_text = AutoMLSearch(X_train=features_train, y_train=target_train,problem_type='binary')
automl_no_text.search()

Using default limit of max_batches=1.

Numerical binary classification target classes must be [0, 1], got [ham, spam] instead
Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: catboost, xgboost, linear_model, lightgbm, extra_trees, decision_tree, random_forest



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 8.638
Batch 1: (2/9) Logistic Regression Classifier w/ Imp... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.561
Batch 1: (3/9) Random Forest Classifier w/ Imputer +... Elapsed:00:06
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.561
Batch 1: (4/9) XGBoost Classifier w/ Imputer + One H... Elapsed:00:09
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.562
Batch 1: (5/9) CatBoost Classifier w/ Imputer           Elapsed:00:09
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.622
Batch 1: (6/9) Elastic Net Classifier w/ Imputer + O... Elapsed:00:11
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.563
Batch 1: (7/9) Extra Trees Classifier w/ Imputer + O... Elapsed:00:11
	Start

In [38]:
best_pipeline_no_text = automl_no_text.best_pipeline
scores = best_pipeline_no_text.score(features_test,target_test,objectives = evalml.objectives.get_core_objectives('binary'))

In [39]:
scores

OrderedDict([('MCC Binary', 0.0710465299061946),
             ('Log Loss Binary', 0.5576891229036224),
             ('AUC', 0.5066740407467751),
             ('Precision', 1.0),
             ('F1', 0.013333333333333332),
             ('Balanced Accuracy Binary', 0.5033557046979866),
             ('Accuracy Binary', 0.7525083612040134)])

In [41]:
automl_no_text.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,1,Logistic Regression Classifier w/ Imputer + On...,0.560630,0.558141,93.509952,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,2,Random Forest Classifier w/ Imputer + One Hot ...,0.561164,0.559126,93.503770,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,6,Extra Trees Classifier w/ Imputer + One Hot En...,0.561178,0.559042,93.503610,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,8,Decision Tree Classifier w/ Imputer + One Hot ...,0.561313,0.560255,93.502042,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,3,XGBoost Classifier w/ Imputer + One Hot Encoder,0.562197,0.561991,93.491811,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,7,LightGBM Classifier w/ Imputer + One Hot Encoder,0.562451,0.561991,93.488872,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,5,Elastic Net Classifier w/ Imputer + One Hot En...,0.562556,0.562070,93.487658,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,4,CatBoost Classifier w/ Imputer,0.622449,0.622812,92.794318,False,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mode Baseline Binary Classification Pipeline,8.638305,8.623860,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}
